In [1]:
import pandas as pd
from pathlib import Path

In [2]:
DATA_DIR = Path("data")
RAW_DIR = DATA_DIR / "raw"

In [18]:
df = pd.read_excel(RAW_DIR / "base_latam-Feb 2023.xlsx")
df.head(10)

C:\Users\daniel.almeida\AppData\Local\pypoetry\Cache\virtualenvs\governments-complying-cfqBn2gm-py3.12\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,ID,CODE,SUBCODE,JURISD,COUNTRY,COUNTRY2,BRANCH,T_ENTITY,LEVEL_GOV,THEME,...,ORIGINAL_LINK,ATRIB_AT,LEVEL_C,LEVEL_B,LEVEL_G,LEVEL,OBS,NEW_LEVEL,Educação,Entidades citadas
0,1,4,NaN,NaN,Dominican Republic,Dominican Republic,Executive,NaN,National,NaN,...,NaN,NaN,0,0.0,0.0,3.0,NaN,3.0,SIM,amet / banco agricola de la republica dominica...
1,2,5,NaN,NaN,Dominican Republic,Dominican Republic,Executive,NaN,National,NaN,...,NaN,NaN,0,0.0,0.0,3.0,NaN,3.0,SIM,amet / banco de reservas / camara de cuentas d...
2,3,6,NaN,NaN,Dominican Republic,Dominican Republic,Executive,NaN,National,NaN,...,NaN,NaN,0,0.0,0.0,3.0,NaN,3.0,SIM,amet / banco agricola de la republica dominica...
3,4,8,NaN,NaN,Ecuador,Ecuador,Executive,NaN,National,Government and policy,...,NaN,NaN,0,0.0,0.0,3.0,NaN,3.0,NÃO,ame / court / lotaip / ministry for policy / m...
4,5,8,NaN,Armenia,Colombia,Ecuador,Executive,NaN,Municipal,NaN,...,NaN,NaN,0,0.0,0.0,4.0,NaN,4.0,NÃO,ame / court / lotaip / ministry for policy / m...
5,6,9,NaN,NaN,Venezuela,Venezuela,"Executive,Judicial,Legislative",Public Defender's Office; Police; Municipal Co...,"National,State-level,Municipal",NaN,...,NaN,NaN,0,1.0,1.0,2.0,NaN,6.0,NÃO,cicpc / conatel / concejo municipal alcaldia d...
6,7,10,NaN,NaN,"Argentina,Bolivia,Chile,Ecuador,Uruguay","Argentina,Bolivia,Chile,Ecuador,Mexico,Peru,Ur...",Executive,NaN,National,NaN,...,NaN,NaN,1,0.0,0.0,1.0,NaN,1.0,NÃO,NaN
7,8,10,1,NaN,Argentina,"Argentina,Bolivia,Chile,Ecuador,Mexico,Peru,Ur...",Executive,NaN,National,NaN,...,NaN,NaN,0,0.0,0.0,3.0,NaN,3.0,NÃO,NaN
8,9,10,2,NaN,Bolivia,"Argentina,Bolivia,Chile,Ecuador,Mexico,Peru,Ur...",Executive,NaN,National,NaN,...,NaN,NaN,0,0.0,0.0,3.0,NaN,3.0,NÃO,NaN
9,10,10,3,NaN,Chile,"Argentina,Bolivia,Chile,Ecuador,Mexico,Peru,Ur...",Executive,NaN,National,NaN,...,NaN,NaN,0,0.0,0.0,3.0,NaN,3.0,NÃO,NaN


In [45]:
copy_df = df.copy()

# Fillin NA's with 0's
copy_df['ID'] = copy_df['ID'].fillna(0)

# Getting rows that cant be converted to int -> they seem to be annotations in the dataset
problematic_rows = copy_df[~copy_df['ID'].apply(lambda x: str(x).isdigit())].index
copy_df = copy_df.drop(problematic_rows)

# With these rows out, we can convert to parquet
copy_df.to_parquet(RAW_DIR / "base-latam.parquet")

ArrowTypeError: ("Expected bytes, got a 'int' object", 'Conversion failed for column ID with type object')

In [44]:
copy_df['SUBCODE'] = copy_df['SUBCODE'].fillna(0)
copy_df['SUBCODE'] = pd.to_numeric(copy_df['SUBCODE'])
problematic_rows = copy_df[~copy_df['SUBCODE'].apply(lambda x: str(x).isdigit())].index
copy_df.iloc[problematic_rows]

ValueError: Unable to parse string "1,1" at position 1556

In [28]:
# Converting column types
for col in copy_df.columns:
    numeric_values = pd.to_numeric(copy_df[col].fillna(0), errors='coerce')
    if not numeric_values.isna().any(): # Se não houver nenhum erro na conversão
        copy_df[col] = numeric_values
    else:
        # Checando se a coluna deve ser convertida para booleano
        unique_values = copy_df[col].unique()
        if set(unique_values) == {'SIM', 'NÃO'}:
            copy_df[col] = copy_df[col].map({'SIM': True, 'NÃO': False})
        else:
            # Leave as str
            copy_df[col] = copy_df[col].astype(str)

In [ ]:
copy_df['ID'] = pd.to_numeric(copy_df['ID'], errors='coerce')
copy_df['CODE'] = pd.to_numeric(copy_df['CODE'])

In [29]:
copy_df.dtypes

ID                     int64
CODE                   int64
SUBCODE               object
JURISD                object
COUNTRY               object
COUNTRY2              object
BRANCH                object
T_ENTITY              object
LEVEL_GOV             object
THEME                 object
METHOD                object
ADM_ITEMS_AT          object
N_ENTITIES_PT         object
DIST_REQ              object
N_REQ                 object
N_RESP                object
N_RESP_ONTIME         object
MEANS_REQ             object
N_PARTIAL_RESP        object
N_PARTIAL_RESP_INV    object
DENIALS               object
DENIALS_MUTE          object
N_REQ_TIME_EXT        object
APPEALS               object
RELATED_AUD           object
N_INST_PT             object
RESP_RATE             object
N_INST_AT             object
COMPLIANCE            object
AUDIT_TYPE            object
COUNTRY_EN            object
COUNTRY_PT            object
COUNTRY_ES            object
REGION                object
TITLE_EN      

In [5]:
df.to_parquet(RAW_DIR / "base-latam.parquet")

ArrowTypeError: ("Expected bytes, got a 'int' object", 'Conversion failed for column ID with type object')